In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [82]:
data = pd.read_csv('stockpile.csv', header=0)

reqcols = list(data.columns)
reqcols.remove('Temperature') # shuffing the order of the columns
reqcols = ['Temperature'] + reqcols
data = data[reqcols]
pts, paras = data.shape
data

,Temperature,Date,Time,wind_direction,Ambient Temperature (in °C),Delta Temp (Y),Log (Y),Temperature - 1 (in °C),Delta Temp (Y-1),North,East,West,South,Wind Impact (Should be linear variation & wind speed),FACE-A,FACE-B,FACE-C
0,44,2018-02-01,10:30:00,165,27,17,2.833213,40,13,0,0,1,0,0.000000,1,0,0
1,51,2018-02-01,10:30:00,165,27,24,3.178054,43,16,0,0,1,0,0.000000,0,1,0
2,53,2018-02-01,10:30:00,165,27,26,3.258097,49,22,0,0,0,1,0.965926,1,0,0
3,61,2018-02-01,10:30:00,165,27,34,3.526361,58,31,0,0,0,1,0.965926,0,1,0
4,65,2018-02-01,10:30:00,165,27,38,3.637586,62,35,0,0,0,1,0.965926,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,41,2018-02-10,15:30:00,180,27,14,2.639057,37,10,0,0,0,1,1.000000,1,0,0
475,35,2018-02-10,15:30:00,180,27,8,2.079442,36,9,0,0,0,1,1.000000,0,1,0
476,32,2018-02-10,15:30:00,180,27,5,1.609438,44,17,0,1,0,0,0.000000,1,0,0
477,32,2018-02-10,15:30:00,180,27,5,1.609438,38,11,1,0,0,0,0.000000,0,1,0


In [83]:
# label_encoder object knows how to understand word labels. 
label_encoder = LabelEncoder() 
unique_times =data['Time'].unique() 
print("initial:", unique_times)
data['Time']= label_encoder.fit_transform(data['Time']) 
print("transformed:", label_encoder.transform(unique_times))

initial: ['10:30:00' '11:00:00' '11:30:00' '12:00:00' '12:30:00' '13:00:00'
 '10:00:00' '14:30:00' '15:00:00' '15:30:00' '13:30:00']
transformed: [ 1  2  3  4  5  6  0  8  9 10  7]


In [84]:
reqcols.remove('Date') # to be used in the next cell
alldays = data.Date.unique()
paras = paras-1 # 1 column is reduced
reqcols

['Temperature',
 'Time',
 'wind_direction',
 'Ambient Temperature (in °C)',
 'Delta Temp (Y)',
 'Log (Y)',
 'Temperature - 1 (in °C)',
 'Delta Temp (Y-1)',
 'North',
 'East',
 'West',
 'South',
 'Wind Impact (Should be linear variation & wind speed)',
 'FACE-A',
 'FACE-B',
 'FACE-C']

In [85]:
stdscl = StandardScaler(copy=False, with_mean = False)
ct = ColumnTransformer([('somename',stdscl, reqcols)], remainder='passthrough')
data = ct.fit_transform(data)

data

array([[3.873603620294357, 0.3156306107959929, 1.9611953701419271, ...,
        0.0, 0.0, '2018-02-01'],
       [4.489858741704823, 0.3156306107959929, 1.9611953701419271, ...,
        2.023638593649286, 0.0, '2018-02-01'],
       [4.665931633536385, 0.3156306107959929, 1.9611953701419271, ...,
        0.0, 0.0, '2018-02-01'],
       ...,
       [2.817166269304987, 3.156306107959929, 2.1394858583366476, ...,
        0.0, 0.0, '2018-02-10'],
       [2.817166269304987, 3.156306107959929, 2.1394858583366476, ...,
        2.023638593649286, 0.0, '2018-02-10'],
       [2.641093377473425, 3.156306107959929, 2.1394858583366476, ...,
        0.0, 2.593778306244962, '2018-02-10']], dtype=object)

In [109]:
# batch_size, epoch and iteration
batch_sz = 16
epochs = 160
seq_ln = 8
#intbatch = batch_sz/seq_ln # remember that seq_ln should divide batch_sz 

In [110]:
strides = 2
newtrm = np.array([]).reshape(0,seq_ln,  paras-1)
newtym = np.array([]).reshape(0,seq_ln)

for day in alldays:
    index = data[:, 16] == day
    ddt = data[index] 
    x = ddt[:, 1:-1]
    y = ddt[:, 0]
    pts = x.shape[0]
    
    newdatasz = (pts - seq_ln)//strides +1
    newtr = np.zeros((newdatasz,seq_ln, paras-1))
    newty = np.zeros((newdatasz,seq_ln))
    print(newdatasz)
    cc = 0
    for i in range(newdatasz):
        newtr[i] = x[cc: cc+seq_ln]
        newty[i] = y[cc: cc+seq_ln]
        cc += strides
        
    newtrm = np.vstack([newtrm, newtr])
    newtym = np.vstack([newtym, newty])
    

27
47
47
22
22
17
17
16


In [5]:
#strides are set to make sequences from 'strides' distance appart
strides = 2
x = data.values[:,0:-1]
y = data.values[:, -1]

newdatasz = (pts - seq_ln)//strides +1
newtrm = np.zeros((newdatasz,seq_ln, paras-1))
newtym = np.zeros((newdatasz,seq_ln))
cc= 0
for i in range(newdatasz):
    newtrm[i] = x[cc: cc+seq_ln]
    newtym[i] = y[cc: cc+seq_ln]
    cc += strides

In [111]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

RNN Model

In [112]:
x_train = torch.FloatTensor(newtrm)
y_train = torch.FloatTensor(newtym)
y_trainlastst = y_train[:, -1] #change possible here

train_data = TensorDataset(x_train,y_trainlastst)
train_loader = DataLoader(train_data,batch_size=batch_sz,shuffle=True)   

In [113]:
# Create RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(RNNModel, self).__init__()  
        # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, 1, batch_first=True, nonlinearity='tanh', dropout=0.2)
        # Readout layer
        self.fc = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.randn(1, x.size(0), self.hidden_dim))
        # rnn n time steps
        out, hn = self.rnn(x, h0)
        #print(out.shape)
        out = self.fc(out[:, -1, :]) 
        return out

In [114]:
model = RNNModel(paras-1, 20)
lossfn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.8, weight_decay=0.05)

C:\Users\DELL\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [115]:
model.eval()
y_pred = model(x_train)

before_train = lossfn(y_pred.squeeze(), y_trainlastst)
print('loss before training' , before_train.item())

loss before training 16.50927734375


In [116]:
model.train()
for epoch in range(epochs):
    loss = 0
    for dx, dy in train_loader:     
        optimizer.zero_grad()
        # Forward pass
        outputs = model(dx)
        outputs = outputs.squeeze()
        # Compute Loss
        c_loss = lossfn(outputs, dy)
        c_loss.backward()
        optimizer.step()
        #print(c_loss.item())
        loss += c_loss.item()
        
    if (epoch % 10 == 0):
        with torch.no_grad():
            y_pred = model(x_train)
            r2acc = r2_score(y_trainlastst, y_pred)
            netloss = loss / len(train_loader)
        print('Epoch: {} :-- loss:= {:.3f} :------------: r2_Accuracy:= {:.3f}'.format(epoch, netloss, r2acc))

Epoch: 0 :-- loss:= 6.323 :------------: r2_Accuracy:= 0.029
Epoch: 10 :-- loss:= 0.149 :------------: r2_Accuracy:= 0.864
Epoch: 20 :-- loss:= 0.086 :------------: r2_Accuracy:= 0.923
Epoch: 30 :-- loss:= 0.062 :------------: r2_Accuracy:= 0.940
Epoch: 40 :-- loss:= 0.056 :------------: r2_Accuracy:= 0.952
Epoch: 50 :-- loss:= 0.047 :------------: r2_Accuracy:= 0.960
Epoch: 60 :-- loss:= 0.036 :------------: r2_Accuracy:= 0.965
Epoch: 70 :-- loss:= 0.033 :------------: r2_Accuracy:= 0.969
Epoch: 80 :-- loss:= 0.028 :------------: r2_Accuracy:= 0.974
Epoch: 90 :-- loss:= 0.025 :------------: r2_Accuracy:= 0.975
Epoch: 100 :-- loss:= 0.020 :------------: r2_Accuracy:= 0.977
Epoch: 110 :-- loss:= 0.026 :------------: r2_Accuracy:= 0.979
Epoch: 120 :-- loss:= 0.019 :------------: r2_Accuracy:= 0.980
Epoch: 130 :-- loss:= 0.018 :------------: r2_Accuracy:= 0.982
Epoch: 140 :-- loss:= 0.020 :------------: r2_Accuracy:= 0.983
Epoch: 150 :-- loss:= 0.016 :------------: r2_Accuracy:= 0.981


In [65]:
model.train()
for epoch in range(10):
    optimizer.zero_grad()
    # Forward pass
    outputs = model(x_train)
    outputs = outputs.squeeze()
    # Compute Loss
    c_loss = lossfn(outputs, y_trainlastst)
    c_loss.backward()
    optimizer.step()
    print(epoch, outputs, c_loss.item())

0 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<SqueezeBackward0>) nan
1 tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [117]:
with torch.no_grad():
    y_pred = model(x_train)
    y_pred = y_pred.squeeze().numpy()
y_tr = y_trainlastst.numpy()
for i in range(len(y_pred)):
    print(y_pred[i], y_tr[i])

4.5818906 4.401822
3.3994255 3.3453848
4.893186 4.8420043
5.6711764 5.722369
5.6762204 5.8104053
4.535112 4.313786
3.4011993 3.3453848
4.6593127 4.4898586
5.73778 5.8104053
5.7320857 5.898442
4.184369 3.9616401
3.3620481 3.3453848
5.454568 5.722369
5.8569465 6.074515
5.7681913 5.9864783
4.16299 4.137713
3.3407888 3.3453848
5.551079 5.8104053
5.9247994 6.2505875
5.780849 6.074515
3.8538775 3.7855673
3.4704537 3.6094942
5.3264213 5.546296
5.716375 5.8104053
5.4578414 5.546296
3.4692998 3.521458
3.1674094 3.2573485
3.9591603 3.8736036
3.1195102 3.0812757
3.6412525 3.6094942
5.0699406 4.930041
4.6413984 4.577895
3.9193192 3.7855673
3.2183957 3.2573485
3.8350472 3.7855673
5.3866315 5.4582596
5.06455 5.106114
3.7980118 3.6975307
3.3333879 3.4334214
3.873982 3.8736036
5.477498 5.546296
5.2342625 5.282187
3.5126195 3.4334214
3.6097078 3.8736036
4.2544513 4.2257495
5.7399316 5.898442
5.335271 5.370223
3.9197693 3.9616401
3.2074537 3.2573485
4.7537136 4.8420043
5.794005 5.9864783
5.187816 5.1941

In [118]:

print ('Average Mean Absolute Error on Test: ', mean_absolute_error(y_tr, y_pred))
print ('Average R^2 Value on Train: ', r2_score(y_tr, y_pred))

Average Mean Absolute Error on Test:  0.096415535
Average R^2 Value on Train:  0.9853970931285204


Keras RNN

In [24]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, GRU, LSTM
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DELL\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [25]:
# custom R2-score metrics for keras backend

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [26]:
kmodel = Sequential()
#Rnn layer
kmodel.add(SimpleRNN(12, activation= 'relu', kernel_initializer='truncated_normal', input_shape=(seq_ln, paras-1)))
#Output Layer
kmodel.add(Dense(1, activation='relu', kernel_initializer='truncated_normal'))


opt = Adam(lr=0.005)
kmodel.compile(loss='mean_squared_error', optimizer=opt, metrics=[r2_keras])
kmodel.summary()





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 12)                276       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [27]:
kmodel.fit(newtr,newty[:, -1],epochs=100,batch_size=32)



Epoch 1/100
107/107 [==============================] - 2s 19ms/step - loss: 1942.5447 - r2_keras: -28.7305
Epoch 2/100
107/107 [==============================] - 0s 447us/step - loss: 1920.9477 - r2_keras: -29.7625
Epoch 3/100
107/107 [==============================] - 0s 457us/step - loss: 1849.8037 - r2_keras: -29.2059
Epoch 4/100
107/107 [==============================] - 0s 419us/step - loss: 1676.6240 - r2_keras: -26.1398
Epoch 5/100
107/107 [==============================] - 0s 447us/step - loss: 1297.6757 - r2_keras: -18.9325
Epoch 6/100
107/107 [==============================] - 0s 410us/step - loss: 604.2467 - r2_keras: -9.0468
Epoch 7/100
107/107 [==============================] - 0s 410us/step - loss: 526.0825 - r2_keras: -6.8953
Epoch 8/100
107/107 [==============================] - 0s 429us/step - loss: 343.3511 - r2_keras: -4.6140
Epoch 9/100
107/107 [==============================] - 0s 419us/step - loss: 318.4481 - r2_keras: -4.5313
Epoch 10/100
107/107 [=============

107/107 [==============================] - 0s 419us/step - loss: 28.5203 - r2_keras: 0.5749
Epoch 76/100
107/107 [==============================] - 0s 429us/step - loss: 28.3863 - r2_keras: 0.5403
Epoch 77/100
107/107 [==============================] - 0s 419us/step - loss: 24.3347 - r2_keras: 0.6042
Epoch 78/100
107/107 [==============================] - 0s 438us/step - loss: 26.5997 - r2_keras: 0.5949
Epoch 79/100
107/107 [==============================] - 0s 438us/step - loss: 25.1059 - r2_keras: 0.6372
Epoch 80/100
107/107 [==============================] - 0s 447us/step - loss: 24.5890 - r2_keras: 0.6213
Epoch 81/100
107/107 [==============================] - 0s 429us/step - loss: 24.7249 - r2_keras: 0.5856
Epoch 82/100
107/107 [==============================] - 0s 419us/step - loss: 25.2606 - r2_keras: 0.5892
Epoch 83/100
107/107 [==============================] - 0s 419us/step - loss: 24.4427 - r2_keras: 0.5974
Epoch 84/100
107/107 [==============================] - 0s 429us/ste

In [30]:
y_pred = kmodel.predict(newtr)
y_act = newty[:, -1]
print ('Average Mean Absolute Error: ', mean_absolute_error(y_act, y_pred))
print ('Average R^2 Value on Train: ', r2_score(y_act, y_pred))

Average Mean Absolute Error:  4.069523050183448
Average R^2 Value on Train:  0.6400566485780137


Keras GRU

In [31]:
gmodel = Sequential()
#Rnn layer
gmodel.add(GRU(12, activation= 'relu', kernel_initializer='truncated_normal', input_shape=(seq_ln, paras-1)))
#Output Layer
gmodel.add(Dense(1, activation='relu', kernel_initializer='random_normal'))


opt = Adam(lr=0.005)
gmodel.compile(loss='mean_squared_error', optimizer=opt, metrics=[r2_keras])
gmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 12)                828       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 841
Trainable params: 841
Non-trainable params: 0
_________________________________________________________________


In [32]:
gmodel.fit(newtr,newty[:, -1],epochs=100,batch_size=32)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
107/107 [==============================] - 4s 34ms/step - loss: 1943.2464 - r2_keras: -29.9231
Epoch 2/100
107/107 [==============================] - 0s 941us/step - loss: 1935.4604 - r2_keras: -28.0592
Epoch 3/100
107/107 [==============================] - 0s 988us/step - loss: 1923.5488 - r2_keras: -31.0667
Epoch 4/100
107/107 [==============================] - 0s 960us/step - loss: 1902.3037 - r2_keras: -27.4627
Epoch 5/100
107/107 [==============================] - 0s 979us/step - loss: 1865.2657 - r2_keras: -28.0700
Epoch 6/100
107/107 [==============================] - 0s 969us/step - loss: 1797.4474 - r2_keras: -26.8168
Epoch 7/100
107/107 [==============================] - 0s 979us/step - loss: 1669.5833 - r2_keras: -24.4897
Epoch 8/100
107/107 [==============================] - 0s 1ms/step - loss: 1424.1671 - r2_keras: -21.5731
Epoch 9/100
107/107 [========================

In [33]:
y_pred = gmodel.predict(newtr)
y_act = newty[:, -1]
print ('Average Mean Absolute Error: ', mean_absolute_error(y_act, y_pred))
print ('Average R^2 Value on Train: ', r2_score(y_act, y_pred))

Average Mean Absolute Error:  2.980531675392222
Average R^2 Value on Train:  0.7854226300513129


Keras LSTM

In [34]:
lmodel = Sequential()
#Rnn layer
lmodel.add(LSTM(12, activation= 'relu', kernel_initializer='truncated_normal', input_shape=(seq_ln, paras-1)))
#Output Layer
lmodel.add(Dense(1, activation='relu', kernel_initializer='random_normal'))


opt = Adam(lr=0.005)
lmodel.compile(loss='mean_squared_error', optimizer=opt, metrics=[r2_keras])
lmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 12)                1104      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13        
Total params: 1,117
Trainable params: 1,117
Non-trainable params: 0
_________________________________________________________________


In [35]:
lmodel.fit(newtr,newty[:, -1],epochs=100,batch_size=32)

Epoch 1/100
107/107 [==============================] - 5s 43ms/step - loss: 1945.2051 - r2_keras: -28.2754
Epoch 2/100
107/107 [==============================] - 0s 1ms/step - loss: 1933.8013 - r2_keras: -28.8019
Epoch 3/100
107/107 [==============================] - 0s 1ms/step - loss: 1900.7029 - r2_keras: -28.2146
Epoch 4/100
107/107 [==============================] - 0s 1ms/step - loss: 1757.3853 - r2_keras: -27.6346
Epoch 5/100
107/107 [==============================] - 0s 1ms/step - loss: 1232.0690 - r2_keras: -19.8464
Epoch 6/100
107/107 [==============================] - 0s 1ms/step - loss: 708.3362 - r2_keras: -10.0129
Epoch 7/100
107/107 [==============================] - 0s 1ms/step - loss: 467.5947 - r2_keras: -6.1673
Epoch 8/100
107/107 [==============================] - 0s 1ms/step - loss: 428.7596 - r2_keras: -5.4341
Epoch 9/100
107/107 [==============================] - 0s 1ms/step - loss: 358.8090 - r2_keras: -4.7782
Epoch 10/100
107/107 [==============================

In [36]:
y_pred = lmodel.predict(newtr)
y_act = newty[:, -1]
print ('Average Mean Absolute Error: ', mean_absolute_error(y_act, y_pred))
print ('Average R^2 Value on Train: ', r2_score(y_act, y_pred))

Average Mean Absolute Error:  4.2723352385904185
Average R^2 Value on Train:  0.5881351591619086
